# Read BVH file ✓

In [2]:
from Motion.BVH import load

animation, joints_names, frametime = load("./test.bvh")

# Write BVH file ✓

In [3]:
from Motion.BVH import save

save(filename="test.bvh", anim=animation, names=joints_names, frametime=frametime)

# Edit BVH file (center animation) ✓

In [4]:
from Motion.BVH import load, save

animation, joints_names, frametime = load("./test.bvh", order="xyz")
animation.positions[:, :, [0, 2]] -= animation.positions[0, 0, [0, 2]]
save(filename="centered.bvh", anim=animation, names=joints_names, frametime=frametime, order="xyz")

# Convert BVH data to XYZ positions ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.BVH import load

animation, joints_name, frametime = load("./test.bvh")
anim_xyz = positions_global(animation)
print(anim_xyz)

# Get kinematic chain from BVH ✓

In [ ]:
from Motion.BVH import load
from Motion.AnimationStructure import get_kinematic_chain

animation, joints_name, frametime = load("./test.bvh")
kinematic_chain = get_kinematic_chain(animation.parents)
print(kinematic_chain)

# Visualize motion ✓

In [ ]:
from Motion.Animation import positions_global
from Motion.AnimationStructure import get_kinematic_chain
from Motion.BVH import load
from Motion.plot_script import plot_3d_motion


animation, joints_name, frametime = load("./test.bvh")
skeleton = get_kinematic_chain(animation.parents)
anim_xyz = positions_global(animation)

plot_3d_motion("test_vis.mp4", skeleton, anim_xyz[:600], title="Test viz", fps=100)

# Convert angles to various representations ✓

Be careful with the order of the axes in the BVH file you're using!!

In [ ]:
import torch
import Motion.transforms as tr
from Motion.Animation import Animation
from Motion.BVH import load, save
from Motion.Quaternions import Quaternions

order = "zyx"
animation, joints_names, frametime = load("test.bvh")
n_frames, n_joints = animation.shape
print(f"Animation loaded\nn frames: {n_frames}\tn_joints: {n_joints}")

# animation.rotations is by default represented as Quaternions
rotations = torch.tensor(animation.rotations.qs)
# print("Quaternions shape:", rotations.shape)

# # # Convert quaternions to 6D representation...
# rotations = tr.quat2repr6d(rotations)
# print("Repr6d shape:", rotations.shape)

# # ... and back to quaternions
# rotations = tr.repr6d2quat(rotations)

# # and now to Euler angles
# rotations = tr.quat2euler(rotations, order="xyz", degrees=False)
# print("Euler shape:", rotations.shape)

# Now export back to BVH... and it should be the exact same :)
new_anim = Animation(
    animation.rotations,
    animation.positions,
    animation.orients,
    animation.offsets,
    animation.parents,
)
save("test_angular_conversions.bvh", animation, joints_names, frametime)

# Import to and from SMPL format

PKL -> BVH

In [ ]:
from Motion.smpl import load_smpl, smpl_to_bvh_data, bvh_data_to_smpl, SMPL_JOINTS_NAMES
from Motion.smpl_utils.utils import bvh

smpl_file = "./test_smpl.pkl"
smpl_dict = load_smpl(smpl_file)

bvh_data = smpl_to_bvh_data(smpl_dict)
bvh.save("smpl2bvh.bvh", bvh_data)

BVH -> PKL

In [ ]:
import numpy as np
from Motion import BVH
from Motion.smpl import load_smpl, smpl_to_bvh_data, bvh_data_to_smpl, SMPL_JOINTS_NAMES
from Motion.smpl_utils.utils import bvh

bvh_file = "test_smpl.bvh"
bvh_data = bvh.load(bvh_file)

smpl_dict = bvh_data_to_smpl(bvh_data)
backtobvhdata = smpl_to_bvh_data(smpl_dict, frametime=1/100)
bvh.save("tothefutureandback.bvh", bvh_data)

To do:
 - [ ] Forward Kinematics :) 
 - [ ] Unify everything. For instance, transforms should accept numpy arrays and not just torch tensors and it should be possible to convert directly from SMPL to Animation object and back.